#### Часть 2. Проверка гипотез методами математической статистики

В этой части группового проекта вам предстоит самим выбрать предметную область, а также самим получить датасет или датасеты, с которыми вы будете работать.

С точки зрения источников данных у вас есть две опции: вы можете найти ваш датасет — или 2-3 датасета в случае необходимости — на сайте [UCI Machine Learning Repository](https://archive.ics.uci.edu/) (брать датасеты необходимо именно с этого сайта); либо же вторая опция — можно собрать датасет/датасеты вручную с помощью средств Parsing'a и API. Допускается использование и комбинации этих двух опций. В любом случае, *каждый* используемый вами датасет должен (до этапа EDA) содержать **не менее 2000 строк** и **не менее 8 признаков**.




После получения датасетов вам следует:
- провести EDA с осмысленными визуализациями и качественным поиском первичных закономерностей;
- на основе проделенного EDA сформулировать **минимум три чёткие, содержательные гипотезы**;
- подобрать и реализовать **минимум три различных статистических теста** для проверки сформулированных раннее гипотез, причём каждый тест должен быть обязательно обоснован (различные статистические тесты в нашем случае означает, что статистические критерии для этих тестов должны быть разными);
- корректно и полно проинтерпретировать полученные результаты и сформулировать итоговые выводы.

**Важно:** вы должны очень ответственно подойти к выбору датасета, так как гипотезы должны быть *не случайными*, а логически вытекающими из осуществленного EDA и, кроме того, имеющими реальный интерес для изучения и проверки с точки зрения бизнес-составляющей. В случае если по одному датасету у вас не получается сформировать 3 достаточно осмысленных гипотезы — разрешается использовать суммарно вплоть до 3 датасетов. В таком случае, все эти датасеты должны иметь какую-то связь между собой.

In [218]:
from selenium import webdriver
import time
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import MoveTargetOutOfBoundsException, StaleElementReferenceException
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin

In [219]:
import time
import re
import pandas as pd

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, WebDriverException
import random


# ====================== НАСТРОЙКА ДРАЙВЕРА ======================

options = webdriver.ChromeOptions()
options.add_argument(
    "user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/131.0.0.0 Safari/537.36"
)

# размер окна поменьше, чтобы меньше всего рисовать
options.add_argument("--window-size=900,700")

# отключаем GPU-ускорение (часто в headless/автоматизации только мешает)
options.add_argument("--disable-gpu")

# не нужны расширения, уведомления, всплывашки
options.add_argument("--disable-extensions")
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")

# чтобы хром меньше душил вкладку, когда она «в фоне»
options.add_argument("--disable-background-timer-throttling")
options.add_argument("--disable-renderer-backgrounding")
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument("--headless=new")

# картинки не грузим вообще — это сильно разгружает карты
prefs = {
    "profile.managed_default_content_settings.images": 2
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)

# ====================== ДАТАФРЕЙМ ======================

columns = [
    'name',
    'link',
    'monWT',
    'tueWT',
    'wedWT',
    'thuWT',
    'friWT',
    'satWT',
    'sunWT',
    'avgBill',
    'rating',
    'rateAmount',
    'address',
    'distMetro',
    'metro',
    'photoAmount',
    'menuPositions',
    "reviewsAmount",
    "cuisine",
    "tablesAmount",
    "phoneListed",
    "reservationButton",
    "linkToSite",
    "categories"

]

df = pd.DataFrame(columns=columns)


# ====================== ХЕЛПЕРЫ ======================
def get_reviews_amount(soup):
    """
    Парсит количество отзывов с вкладки 'Отзывы'.

    Ищем таб вида:
      <div class="tabs-select-view__title_name_reviews"
           aria-label="Отзывы 25" ...>

    Возвращаем int (например, 25) или None, если ничего не нашли.
    """
    # пытаемся поймать по aria-label или по классу
    el = soup.select_one(
        'div[role="tab"][aria-label^="Отзывы"], '
        'div.tabs-select-view__title_name_reviews'
    )
    if not el:
        return None

    # берём либо aria-label, либо видимый текст
    label = el.get("aria-label") or el.get_text(strip=True)
    return _clean_int(label)

def _clean_int(text):
    if not text:
        return None
    text = text.replace("\u00a0", " ")
    m = re.search(r"\d+", text)
    return int(m.group()) if m else None


def _format_avg_bill_raw(text):
    """
    '1 000 ₽'       -> '1000'
    '1 000–1 500 ₽' -> '1000-1500'
    """
    if not text:
        return None
    text = text.replace("\u00a0", " ")
    nums = re.findall(r"\d+", text)
    if not nums:
        return None
    if len(nums) == 1:
        return nums[0]
    return f"{nums[0]}-{nums[1]}"


# ====================== ПОЛЯ ИЗ soup ======================

def get_name(soup):
    h1 = soup.select_one('h1[itemprop="name"]')
    if h1 and h1.contents:
        return h1.contents[0].strip()
    return None


def get_rating(soup):
    el = soup.select_one("span.business-rating-badge-view__rating-text")
    if not el:
        return None
    txt = el.get_text(strip=True).replace(",", ".")
    try:
        return float(txt)
    except ValueError:
        return None


def get_rate_amount(soup):
    el = soup.select_one("div.business-header-rating-view__text._clickable")
    if not el:
        return 0

    aria = el.get("aria-label") or ""
    val = _clean_int(aria)
    if val is not None:
        return val

    txt = el.get_text(strip=True)
    return _clean_int(txt)


def get_address(soup):
    el = soup.select_one("a.business-contacts-view__address-link")
    return el.get_text(strip=True) if el else None


def get_metro_name(soup):
    block = soup.select_one("div.masstransit-stops-view__stop._type_metro")
    if not block:
        return None
    name_el = block.select_one("div.masstransit-stops-view__stop-name")
    return name_el.get_text(strip=True) if name_el else None


def get_dist_metro(soup):
    block = soup.select_one("div.masstransit-stops-view__stop._type_metro")
    if not block:
        return None

    dist_el = block.select_one("div.masstransit-stops-view__stop-distance-text")
    if not dist_el:
        return None

    # берём текст как есть и чуть приводим в порядок
    txt = dist_el.get_text(strip=True)
    # убираем неразрывные пробелы, делаем обычные
    txt = txt.replace("\u00a0", " ")

    # теперь txt будет вида "170 м" или "1,2 км"
    return txt


def has_menu(soup):
    if soup.select_one("div.tabs-select-view__title._name_menu"):
        return True
   
    return False


def get_avg_bill(soup):
    for block in soup.select("div.business-features-view__valued"):
        title_el = block.select_one("span.business-features-view__valued-title")
        if not title_el:
            continue
        title_txt = title_el.get_text(strip=True)
        if "Средний счёт" not in title_txt:
            continue

        value_el = block.select_one("span.business-features-view__valued-value")
        if not value_el:
            return None
        value_txt = value_el.get_text(strip=True)
        return _format_avg_bill_raw(value_txt)
    return None


def get_photo_amount(soup):
    """
    Ищем таб 'Фото 53' и вытаскиваем число.
    """
    el = soup.select_one('div[role="tab"][aria-label^="Фото"]')
    if not el:
        el = soup.select_one("div.tabs-select-view__title_name_media")
    if not el:
        # запасной вариант: берём любой таб, где в aria-label есть 'Фото'
        for cand in soup.select('div[role="tab"]'):
            label = cand.get("aria-label", "")
            if "Фото" in label:
                el = cand
                break
    if not el:
        return 0

    label = el.get("aria-label") or el.get_text(strip=True)
    return _clean_int(label)


# ====================== ГРАФИК РАБОТЫ ======================

def get_html_worktime(driver):
    """
    Пытается открыть попап 'График работы' и вернуть html.
    Если элемента нет — возвращает None.
    """
    try:
        time.sleep(random.uniform(1, 2))
        elem = driver.find_element(By.CSS_SELECTOR, "div.business-working-status-flip-view__control")

        actions = ActionChains(driver)
        actions.move_to_element(elem).click().perform()
        time.sleep(random.uniform(2, 3))

        new_html = driver.page_source

        actions = ActionChains(driver)
        actions.send_keys(Keys.ESCAPE).perform()

        return new_html
    except NoSuchElementException:
        return None
    except WebDriverException:
        return None


def parse_working_time(soup):
    """
    Возвращает dict с ключами monWT..sunWT (строки или None).
    """
    day_map = {
        "Понедельник": "monWT",
        "Вторник": "tueWT",
        "Среда": "wedWT",
        "Четверг": "thuWT",
        "Пятница": "friWT",
        "Суббота": "satWT",
        "Воскресенье": "sunWT",
    }

    res = {col: None for col in day_map.values()}

    if soup is None:
        return res

    for row in soup.select("div.business-working-intervals-view__item"):
        day_el = row.select_one("div.business-working-intervals-view__day")
        if not day_el:
            continue
        day_name = day_el.get_text(strip=True)

        col_name = day_map.get(day_name)
        if not col_name:
            continue

        intervals_el = row.select_one("div.business-working-intervals-view__intervals")
        if not intervals_el:
            res[col_name] = None
        else:
            intervals_text = intervals_el.get_text(" ", strip=True)
            res[col_name] = intervals_text

    return res


# ====================== МЕНЮ ======================

def get_menu_items_count(driver):
    """
    Кликает по вкладке 'Меню' и возвращает количество блюд.
    Если вкладки нет/ошибка — 0.
    """
    menu_tab = driver.find_element(
            By.CSS_SELECTOR,
            'div[role="tab"][aria-label^="Меню"], '   # aria-label="Меню, 25"
            'div.tabs-select-view__title._name_menu'  # классы tabs-select-view__title _name_menu
        )
    actions = ActionChains(driver)
    time.sleep(random.uniform(1, 2))
    actions.move_to_element(menu_tab).click().perform()
    time.sleep(random.uniform(1, 2))

    dishes = driver.find_elements(By.CSS_SELECTOR, "div.related-item-photo-view__main")
    return len(dishes)



def open_features_and_get_soup(driver):
    """
    Кликает по вкладке 'Особенности' и возвращает BeautifulSoup
    для обновлённой страницы.
    Если вкладку не нашёл/ошибка — возвращает None.
    """
    try:
        # таб "Особенности": либо по aria-label, либо по классу
        features_tab = driver.find_element(
            By.CSS_SELECTOR,
            'div[role="tab"][aria-label^="Особенности"], ''div.tabs-select-view__title_name_features')
        actions = ActionChains(driver)
        time.sleep(1)
        actions.move_to_element(features_tab).click().perform()
        time.sleep(random.uniform(1, 2))  # даём вкладке прогрузиться

        html = driver.page_source
        ActionChains(driver).send_keys(Keys.ESCAPE).perform()
        soup = BeautifulSoup(html, "html.parser")
        return soup
    except:
        return None


def get_feature_value(soup, label_part):
    """
    Ищет в 'Особенностях' строку, в заголовке которой есть label_part
    ('кухня', 'количество столов' и т.п.), и возвращает её значение.
    Если нет — None.
    """
    if soup is None:
        return None

    # каждый ряд вида: [Заголовок] [Значение]
    for block in soup.select("div.business-features-view__valued"):
        title_el = block.select_one(".business-features-view__valued-title")
        value_el = block.select_one(".business-features-view__valued-value")
        if not title_el or not value_el:
            continue

        title_text = title_el.get_text(" ", strip=True).lower()
        if label_part.lower() in title_text:
            return value_el.get_text(" ", strip=True)

    return None


def get_cuisine(soup):
    # 'Кухня: грузинская, европейская, кавказская, ...'
    return get_feature_value(soup, "кухня")


def get_tables_seats(soup):
    # 'Количество столов в зале для посадки: 20–23' или 'от 40'
    return get_feature_value(soup, "количество столов")

def has_show_phone_button(soup):
    """
    Проверяет, есть ли на странице кнопка 'Показать телефон'.
    На вход получает BeautifulSoup, возвращает True/False.
    """
    if soup is None:
        return False

    btn = soup.find(
        "span",
        string=lambda s: s and "Показать телефон" in s
    )
    return btn is not None

def has_reservation_button(soup):
    """
    True, если где-то на странице есть текст 'забронировать столик'.
    """
    if soup is None:
        return False

    page_text = soup.get_text(" ", strip=True).lower()
    return "забронировать столик" in page_text

def has_internet_button(soup):
    """
    Проверяет, есть ли на карточке кнопка перехода на сайт
    (та самая с классом action-button-view__type_web).
    На вход — BeautifulSoup. Возвращает True / False.
    """
    if soup is None:
        return False

    return soup.select_one(".action-button-view__type_web") is not None

def get_categories(soup):
    """
    Возвращает строку вида:
    'Ресторан, Кафе, Бар'
    или None, если категорий нет.
    """

    if soup is None:
        return None

    # 1) Новый формат: orgpage-categories-info-view
    block = soup.select_one("div.orgpage-categories-info-view")
    if block:
        names = []
        for span in block.select("span.button__text"):
            txt = span.get_text(strip=True)
            if txt:
                names.append(txt)
        if names:
            return ", ".join(names)

    # 2) На всякий случай оставим поддержку старого business-categories-view
    block = soup.select_one("div.business-categories-view")
    if block:
        names = []
        for a in block.select("a.business-categories-view__category"):
            txt = a.get_text(strip=True)
            if txt:
                names.append(txt)
        if names:
            return ", ".join(names)

    return None

# ====================== ГЛАВНАЯ ФУНКЦИЯ ======================

def get_info(url):
    """
    Парсит одну карточку ресторана и добавляет строку в df.
    Возвращает dict с данными по этому ресторану.
    """
    global df

    driver.get(url)
    time.sleep(random.uniform(3, 4))

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    
    wt_html = get_html_worktime(driver)
    wt_soup = BeautifulSoup(wt_html, "html.parser") if wt_html else None
    wt = parse_working_time(wt_soup)

    osobennosti_soup = open_features_and_get_soup(driver)
    # меню
    if has_menu(soup):
        menu_positions = get_menu_items_count(driver)
    else:
        menu_positions = 0

    row = {
        "name": get_name(soup),
        "link": url,
        "monWT": wt.get("monWT"),
        "tueWT": wt.get("tueWT"),
        "wedWT": wt.get("wedWT"),
        "thuWT": wt.get("thuWT"),
        "friWT": wt.get("friWT"),
        "satWT": wt.get("satWT"),
        "sunWT": wt.get("sunWT"),
        "avgBill": get_avg_bill(soup),
        "rating": get_rating(soup),
        "rateAmount": get_rate_amount(soup),
        "address": get_address(soup),
        "distMetro": get_dist_metro(soup),
        "metro": get_metro_name(soup),
        "photoAmount": get_photo_amount(soup),
        "menuPositions": menu_positions,
        "reviewsAmount": get_reviews_amount(soup),
        "cuisine": get_cuisine(osobennosti_soup),
        "tablesAmount": get_tables_seats(osobennosti_soup),
        "phoneListed": has_show_phone_button(soup),
        "reservationButton": has_reservation_button(soup),
        "linkToSite": has_internet_button(soup),
        "categories": get_categories(osobennosti_soup)
    }

    df.loc[len(df)] = row
    return row


# ====================== ПРИМЕР ЗАПУСКА ======================

# mas = [
#     "https://yandex.ru/maps/org/ambassadoriya/101555527161/",
#     "https://yandex.ru/maps/org/ararat/10720990466/",
#     "https://yandex.ru/maps/org/archie/121738924801/",
#     "https://yandex.ru/maps/org/armeniya/1133929046/",
#     "https://yandex.ru/maps/org/armyanskiy_dom/166903702287/",
# ]
#
# for url in mas:
#     row = get_info(url)
#     print(row)
#
# print(df.head())
# driver.quit()

In [216]:
toRead = pd.read_csv("final_df_links_part1.csv")




In [217]:
toRead = toRead["url"]
toRead

0       https://yandex.ru/maps/org/abu_gosh/238185678950/
1       https://yandex.ru/maps/org/al33_pitstseriya/17...
2       https://yandex.ru/maps/org/ambassadori_trampli...
3        https://yandex.ru/maps/org/amphora/128434812078/
4       https://yandex.ru/maps/org/anderson/104269030501/
                              ...                        
2350    https://yandex.com/maps/org/china_today/113703...
2351    https://yandex.com/maps/org/coffeemania/102595...
2352    https://yandex.com/maps/org/coffeemania/104660...
2353    https://yandex.com/maps/org/coffeemania/106931...
2354    https://yandex.com/maps/org/coffeemania/107517...
Name: url, Length: 2355, dtype: object

In [168]:
for i in toRead:
    raw_data = get_info(i)
    print(raw_data)

df.to_csv("processedRestaurants", index=False)

{'name': '1 Этаж', 'link': 'https://yandex.ru/maps/org/1_etazh/1011263209/', 'monWT': '10:00–23:00', 'tueWT': '10:00–23:00', 'wedWT': '10:00–23:00', 'thuWT': '10:00–23:00', 'friWT': '10:00–23:00', 'satWT': '10:00–23:00', 'sunWT': '10:00–23:00', 'avgBill': '500-1000', 'rating': 4.2, 'rateAmount': 347, 'adress': 'ул. Костякова, 6/5, Москва', 'distMetro': 770, 'metro': 'Дмитровская', 'photoAmount': 84, 'menuPositions': 0, 'reviewsAmount': 178}
{'name': '16 Тонн', 'link': 'https://yandex.ru/maps/org/16_tonn/186517304540/', 'monWT': '12:00–05:00', 'tueWT': '12:00–05:00', 'wedWT': '12:00–05:00', 'thuWT': '12:00–05:00', 'friWT': '12:00–05:00', 'satWT': '12:00–05:00', 'sunWT': '12:00–05:00', 'avgBill': None, 'rating': 4.7, 'rateAmount': 4537, 'adress': 'ул. Арбат, 28/1с1, Москва', 'distMetro': 600, 'metro': 'Смоленская', 'photoAmount': 146, 'menuPositions': 100, 'reviewsAmount': 2050}
{'name': 'Км20 Food', 'link': 'https://yandex.com/maps/org/20_food/188515485499/', 'monWT': '11:00–23:00', 'tu

/var/folders/vc/wclpgn_s2xddbp_wllkt2tpc0000gn/T/ipykernel_19696/3297854766.py:356: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = row


{'name': 'Asia mini', 'link': 'https://yandex.com/maps/org/asia_mini/154916420332/', 'monWT': None, 'tueWT': None, 'wedWT': None, 'thuWT': None, 'friWT': None, 'satWT': None, 'sunWT': None, 'avgBill': None, 'rating': None, 'rateAmount': None, 'adress': 'ул. Лавриненко, 11, корп. 1, Москва', 'distMetro': 1, 'metro': 'Некрасовка', 'photoAmount': None, 'menuPositions': 40, 'reviewsAmount': None}
{'name': 'Asian street food', 'link': 'https://yandex.com/maps/org/asian_street_food/191193798644/', 'monWT': '10:00–22:00', 'tueWT': '10:00–22:00', 'wedWT': '10:00–22:00', 'thuWT': '10:00–22:00', 'friWT': '10:00–23:00', 'satWT': '10:00–23:00', 'sunWT': '10:00–22:00', 'avgBill': '500', 'rating': 4.8, 'rateAmount': 22, 'adress': 'Автозаводская ул., 18, Москва', 'distMetro': 930, 'metro': 'ЗИЛ', 'photoAmount': 25, 'menuPositions': 53, 'reviewsAmount': 21}
{'name': 'Askaneli', 'link': 'https://yandex.com/maps/org/askaneli/126128575691/', 'monWT': '12:00–00:00', 'tueWT': '12:00–00:00', 'wedWT': '12:00

/var/folders/vc/wclpgn_s2xddbp_wllkt2tpc0000gn/T/ipykernel_19696/3297854766.py:356: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = row


{'name': 'Bo Cha', 'link': 'https://yandex.com/maps/org/bo_cha/148137327402/', 'monWT': '10:00–22:00', 'tueWT': '10:00–22:00', 'wedWT': '10:00–22:00', 'thuWT': '10:00–22:00', 'friWT': '10:00–22:00', 'satWT': '10:00–22:00', 'sunWT': '10:00–22:00', 'avgBill': None, 'rating': None, 'rateAmount': None, 'adress': 'Большая Тульская ул., 13, Москва', 'distMetro': 70, 'metro': 'Тульская', 'photoAmount': 5, 'menuPositions': 'негры', 'reviewsAmount': None}
{'name': 'Bo food', 'link': 'https://yandex.com/maps/org/bo_food/165471844413/', 'monWT': '10:00–21:00', 'tueWT': '10:00–21:00', 'wedWT': '10:00–21:00', 'thuWT': '10:00–21:00', 'friWT': '10:00–21:00', 'satWT': '10:00–21:00', 'sunWT': '10:00–21:00', 'avgBill': None, 'rating': 4.3, 'rateAmount': 23, 'adress': 'Боровая ул., 10, корп. 1, Москва', 'distMetro': 1, 'metro': 'Лефортово', 'photoAmount': 25, 'menuPositions': 'негры', 'reviewsAmount': 16}
{'name': 'Bo Food', 'link': 'https://yandex.com/maps/org/bo_food/50170713442/', 'monWT': '10:00–22:0

...

{'name': 'ВкусВилл', 'link': 'https://yandex.com/maps/org/vkusvill/58531020745/', 'monWT': '08:00–22:00', 'tueWT': '08:00–22:00', 'wedWT': '08:00–22:00', 'thuWT': '08:00–22:00', 'friWT': '08:00–22:00', 'satWT': '08:00–22:00', 'sunWT': '08:00–22:00', 'avgBill': None, 'rating': None, 'rateAmount': None, 'adress': 'Мясницкая ул., 17, стр. 1, Москва', 'distMetro': 227, 'metro': 'Тургеневская', 'photoAmount': 16, 'menuPositions': 546, 'reviewsAmount': 1}
{'name': 'ВкусВилл', 'link': 'https://yandex.com/maps/org/vkusvill/8989236288/', 'monWT': '08:00–22:00', 'tueWT': '08:00–22:00', 'wedWT': '08:00–22:00', 'thuWT': '08:00–22:00', 'friWT': '08:00–22:00', 'satWT': '08:00–22:00', 'sunWT': '08:00–22:00', 'avgBill': None, 'rating': 4.4, 'rateAmount': 1186, 'adress': 'Михалковская ул., 20, Москва', 'distMetro': 580, 'metro': 'Коптево', 'photoAmount': 38, 'menuPositions': 'негры', 'reviewsAmount': 352}
{'name': 'ВкусВилл', 'link': 'https://yandex.com/maps/org/vkusvill/91789873425/', 'monWT': '08:00–

/var/folders/vc/wclpgn_s2xddbp_wllkt2tpc0000gn/T/ipykernel_19696/3297854766.py:356: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = row


{'name': 'ВкусВилл, кафе быстрого обслуживания', 'link': 'https://yandex.com/maps/org/vkusvill_kafe_bystrogo_obsluzhivaniya/40484219517/', 'monWT': '08:00–22:00', 'tueWT': '08:00–22:00', 'wedWT': '08:00–22:00', 'thuWT': '08:00–22:00', 'friWT': '08:00–22:00', 'satWT': '10:00–21:00', 'sunWT': '10:00–21:00', 'avgBill': None, 'rating': None, 'rateAmount': None, 'adress': '1-й Смоленский пер., 21, Москва', 'distMetro': 221, 'metro': 'Смоленская', 'photoAmount': 7, 'menuPositions': 562, 'reviewsAmount': None}
{'name': 'Владимир', 'link': 'https://yandex.com/maps/org/vladimir/1043573167/', 'monWT': '07:00–11:00', 'tueWT': '07:00–11:00', 'wedWT': '07:00–11:00', 'thuWT': '07:00–11:00', 'friWT': '07:00–11:00', 'satWT': '07:00–11:30', 'sunWT': '07:00–11:30', 'avgBill': '1500-3000', 'rating': 4.3, 'rateAmount': 25, 'adress': '1-я Тверская-Ямская ул., 19, Москва', 'distMetro': 450, 'metro': 'Белорусская', 'photoAmount': 24, 'menuPositions': 'негры', 'reviewsAmount': 8}
{'name': 'Vоdный', 'link': 'h

/var/folders/vc/wclpgn_s2xddbp_wllkt2tpc0000gn/T/ipykernel_19696/3297854766.py:356: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df.loc[len(df)] = row


{'name': 'Вьет Бак вьетнамская кухня', 'link': 'https://yandex.com/maps/org/vyet_bak_vyetnamskaya_kukhnya/184061972095/', 'monWT': None, 'tueWT': None, 'wedWT': None, 'thuWT': None, 'friWT': None, 'satWT': None, 'sunWT': None, 'avgBill': None, 'rating': None, 'rateAmount': None, 'adress': 'Тишинская площадь, 1, стр. 1, Москва', 'distMetro': 970, 'metro': 'Белорусская', 'photoAmount': 4, 'menuPositions': 33, 'reviewsAmount': 1}
{'name': 'Вьет Лотос', 'link': 'https://yandex.com/maps/org/vyet_lotos/184108351201/', 'monWT': '09:00–22:00', 'tueWT': '09:00–22:00', 'wedWT': '09:00–22:00', 'thuWT': '09:00–22:00', 'friWT': '09:00–22:00', 'satWT': '09:00–22:00', 'sunWT': '09:00–22:00', 'avgBill': None, 'rating': 4.4, 'rateAmount': 40, 'adress': 'Ленинградский просп., 62А, Москва', 'distMetro': 45, 'metro': 'Аэропорт', 'photoAmount': 33, 'menuPositions': 48, 'reviewsAmount': 33}
{'name': 'Вьет Вок', 'link': 'https://yandex.ru/maps/org/vyet_vok/212688065559/', 'monWT': '10:00–23:00', 'tueWT': '10

In [175]:
df[df["reviewsAmount"] != "None"].sort_values(by="reviewsAmount")

,name,link,monWT,tueWT,wedWT,thuWT,friWT,satWT,sunWT,avgBill,rating,rateAmount,adress,distMetro,metro,photoAmount,menuPositions,reviewsAmount
2687,Вьет Бак вьетнамская кухня,https://yandex.com/maps/org/vyet_bak_vyetnamsk...,None,None,None,None,None,None,None,None,NaN,None,"Тишинская площадь, 1, стр. 1, Москва",970,Белорусская,4,33,1
2668,ВкусВилл,https://yandex.com/maps/org/vkusvill/58531020745/,08:00–22:00,08:00–22:00,08:00–22:00,08:00–22:00,08:00–22:00,08:00–22:00,08:00–22:00,None,NaN,None,"Мясницкая ул., 17, стр. 1, Москва",227,Тургеневская,16,546,1
707,Foodband,https://yandex.com/maps/org/foodband/199611813...,None,None,None,None,None,None,None,None,NaN,None,"ул. Плющева, 18, корп. 2, Москва",2,Шоссе Энтузиастов,6,213,1
857,Гурмания,https://yandex.com/maps/org/gurmaniya/72871233...,None,None,None,None,None,None,None,None,NaN,None,"Керамическая ул., 16, Балашиха",None,None,None,187,1
2532,TuTon вьетнамская и тайская кухня,https://yandex.com/maps/org/tuton_vyetnamskaya...,None,None,None,None,None,None,None,None,NaN,None,"Рязанский просп., 2, корп. 2, Москва",195,Нижегородская,3,31,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,Foodband,https://yandex.com/maps/org/foodband/187003798...,None,None,None,None,None,None,None,None,NaN,None,"Новоалексеевская ул., 12, стр. 1, Москва",500,Алексеевская,None,221,None
894,IceYou,https://yandex.com/maps/org/iceyou/114248051919/,None,None,None,None,None,None,None,None,NaN,None,"Мясницкая ул., 24/7с3, Москва",380,Тургеневская,None,4,None
1454,Море Рыбы,https://yandex.ru/maps/org/more_ryby/79310019304/,None,None,None,None,None,None,None,None,2.5,7,"Вольная ул., 30, стр. 1, Москва",1,Соколиная Гора,None,негры,None
1969,Too Much,https://yandex.com/maps/org/restoran_too_much/...,12:00–00:00,12:00–00:00,12:00–00:00,12:00–00:00,12:00–00:00,12:00–00:00,12:00–00:00,1000-1500,4.7,None,"Малый Гнездниковский пер., 12, Москва",228,Тверская,11,0,None
